In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns


# Input data files are available in the "../input/" directory.

# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

%matplotlib inline

# Any results you write to the current directory are saved as output.

# Data loading 

In [ ]:
dftrain = pd.read_csv("../input/train.csv")
dftest = pd.read_csv("../input/test.csv")


In [ ]:
dftrain.head()

In [ ]:
dftrain.tail()

In [ ]:
dftest.head()

In [ ]:
dftest.tail()

# Data exploration

In [ ]:
dftrain.info()

### ** >  ce Data ( train) de 11 colonnes est composé de 64 floats, 64 int et  4 object **
---- 


In [ ]:
dftest.info()

### ** >  ce Data ( test) de 9 colonnes est composé de 64 floats, 64 int et  3 object **
---- 

In [ ]:
dftrain.duplicated().sum()

### ** >  il n'y à pas de valeur dupliquées **
---- 

In [ ]:
dftest.duplicated().sum()

### ** >  il n'y à pas de valeur dupliquées pour le test non plus **
---- 

In [ ]:
dftrain.isna().sum()

### ** >  il n'y à pas de valeur manquantes **
---- 


In [ ]:
dftest.isna().sum()

### ** >  il n'y à pas de valeur manquantes pour le test non plus **
---- 

# Data Processing

### ** >  etant donné que ce sont les valeurs dutrain qui m'interessent pour traveiller, je fais un describe afin d'avoir plus de visibilité sur toutes les valeurs moyennes ( qui vont me permettre de determiner un intervalle par la suite )  **
---- 

In [ ]:
dftrain.describe().T

### ** >   un trajet moyen est d'environ 960 secondes , soit 16 minutes .si l'on se base que sur la moyenne , une grande partie des données sera negligées . Pour cela je considére qu'un trajet en taxi peut se faire à partir de 300 secondes soit  5 min,  jusqu'à 86400 secondes soit 24h ( car un taxi peut etre loué pour une journée entiere **
   ---- 


In [ ]:
dftrain = dftrain[dftrain['trip_duration'] > 60 ]
dftrain = dftrain[dftrain['trip_duration'] < 8640 ]



In [ ]:
dftrain.count()

### ** >  123724 données  representent une bonne partie de mon dataset que je considére sufisants pour la suite de mon analyse **
---- 

In [ ]:
plt.figure(figsize=(16, 10))
dftrain["trip_duration_log"]= np.log(dftrain["trip_duration"].values+ 1)
plt.hist(dftrain["trip_duration_log"], bins=100)
plt.xlabel("trip_duration")
plt.ylabel("number of train records")
plt.title("trip duration repartition")
plt.grid(True)
plt.show()

### ** > cette courbe ( loi ) permet d'afficher la trip  duration en fonction du nombre **
---- 

In [ ]:
dftrain = dftrain[dftrain['pickup_longitude'] <= -73.75]
dftrain = dftrain[dftrain['pickup_longitude'] >= -74.03]
dftrain = dftrain[dftrain['pickup_latitude'] <= 40.85]
dftrain = dftrain[dftrain['pickup_latitude'] >= 40.63]
dftrain = dftrain[dftrain['dropoff_longitude'] <= -73.75]
dftrain = dftrain[dftrain['dropoff_longitude'] >= -74.03]
dftrain = dftrain[dftrain['dropoff_latitude'] <= 40.85]
dftrain = dftrain[dftrain['dropoff_latitude'] >= 40.63]

In [ ]:
dftrain.shape

### ** >  le pickup et dropoff  longitude et lattitude sont des parametres qui meritent d'etre cernés afin d'avoir le nombre de  données optimales pour la suite de l'analyse **
---- 

In [ ]:
dftrain = dftrain[dftrain['passenger_count']<= 9 ]
dftrain = dftrain[dftrain['passenger_count']>= 1 ]

In [ ]:
dftrain.shape

### ** >  le nombre de passager egalement est un parametre interessant à analysé, pour ecarter toutes les donnés ne comportant pas de passager et donc inutiles **

---- 

In [ ]:
f, ax = plt.subplots(figsize=(10.5, 10.5))
sns.set(style = "darkgrid")
sns.lineplot(x= "passenger_count", y="trip_duration",
             data=dftrain);

### ** >  ceci explique l'evolution de la relation nombre de passager et durée de trip **
---- 

In [ ]:
f, ax = plt.subplots( figsize=(10.5, 10.5))
sns.despine(f, left=True, bottom=True)
sns.scatterplot(x="passenger_count", y="trip_duration",
               data=dftrain)

### ** >  on y remarque une concentration de passager en fonction de la durée ( 1 passager etant le plus commun )**
---- 

In [ ]:
dftrain["pickup_datetime"] = pd.to_datetime(dftrain["pickup_datetime"], format="%Y-%m-%d %H:%M:%S", utc=True)
dftrain["dropoff_datetime"] = pd.to_datetime(dftrain["dropoff_datetime"], format="%Y-%m-%d %H:%M:%S", utc=True)

### ** >  toujours dans un cadre d'optimisation de donnée pour de meilleurs resulats, je trouvais necessaire de transformer le format du pickup et dropoff datetime pour plus de visibilité **
---- 

In [ ]:
dftrain['month'] = dftrain['pickup_datetime'].dt.month_name()

In [ ]:

dftrain.groupby(['month'])['month'].count().plot(kind='bar', figsize=(15, 5));

### ** >  l'affichage des mois peut nous renseigner sur les periodes à fort flux , et l'on remarque une baisse au mois de janvier : cela peut il etre expliqué par la saison  hivernale ?  **
---- 

In [ ]:
dftrain['hours'] = dftrain['pickup_datetime'].dt.hour
dftrain.groupby(['hours'])['trip_duration'].count().plot(kind='bar', figsize=(15, 5));

### ** >  cet affichage nous renseigne sur les heures  à fort flux , et l'on remarque une forte augmentation à 18h et 19h , ce qui peut etre expliqué par la fin de journée de travail  **

In [ ]:
dftrain['days'] = dftrain['pickup_datetime'].dt.day
dftrain.groupby(['days'])['trip_duration'].count().plot(kind='bar', figsize=(15, 5));

### ** >  cet affichage nous renseigne sur les jours   à fort flux . il n y à pas de difference remarquable entre les jours, sauf pour le 31, cela peut il etre expliqué par le jour en question ? ( c'est peut etre un dimanche ) ou un porte monaie vide vu que c'est le dernier jour du mois ? l  **

In [ ]:
# ML librairies 
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

# Features engineering

In [ ]:
dftrain['pickup_datetime'] = pd.to_datetime(dftrain['pickup_datetime'])
dftest['pickup_datetime'] = pd.to_datetime(dftest['pickup_datetime'])

dftrain['month'] = dftrain['pickup_datetime'].dt.month
dftrain['day'] = dftrain['pickup_datetime'].dt.day
dftrain['hour'] = dftrain['pickup_datetime'].dt.hour
dftrain['minute'] = dftrain['pickup_datetime'].dt.minute

dftest['month'] = dftest['pickup_datetime'].dt.month
dftest['day'] = dftest['pickup_datetime'].dt.day
dftest['hour'] = dftest['pickup_datetime'].dt.hour
dftest['minute'] = dftest['pickup_datetime'].dt.minute

dftrain.head()

In [ ]:
select_train = ["passenger_count", "pickup_longitude", "pickup_latitude", "dropoff_longitude","dropoff_latitude","month", "day", "hour", "minute"]
select_test = ["passenger_count", "pickup_longitude", "pickup_latitude", "dropoff_longitude","dropoff_latitude", "month", "day", "hour", "minute"]


X_train = dftrain[select_train]
y_train = dftrain["trip_duration"] 
X_test = dftest[select_test]

In [ ]:
X_train.shape, X_test.shape, y_train.shape

# Validation 

In [ ]:
m = RandomForestRegressor(min_samples_leaf=3)
##print(" cross validation ")#
#m_scores = cross_val_score(m, X_train, y_train, cv=5, scoring ="neg_mean_squared_log_error")
#for i in range(len(m_scores)):
    
 #   m_scores[i] = np.sqrt(abs(m_scores[i]))
#print(m_scores)
#pd.DataFrame(m_scores).mean(), pd.DataFrame(m_scores).std()

In [ ]:
print("training")
m.fit(X_train, y_train)

# Prediction

In [ ]:
pred = m.predict(X_test)
pred

# Soumission

In [ ]:
myriam_submit = pd.DataFrame({"id" : dftest.id,"trip_duration" : pred})

myriam_submit.head()
myriam_submit.to_csv("submit_file.csv", index=False)